In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('../data/hmeq_clean.csv')
y = data['BAD']

In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model_rf = RandomForestClassifier(n_estimators=200)
model_rf.fit(data.drop(columns=['BAD','Unnamed: 0']), y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [122]:
data_pos = data.loc[data['BAD']==0]
data_pos.drop(columns=['BAD','Unnamed: 0'],inplace=True)
data_pos = data_pos.T

/Users/luciasuelves/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [123]:
from scipy.spatial.distance import pdist, squareform

In [124]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pos.T, 'euclidean'))), 
                         index=data_pos.columns, columns=data_pos.columns)

distances.head()

,4,13,19,26,29,30,34,35,38,48,...,5950,5951,5952,5953,5954,5955,5956,5957,5958,5959
4,1.000000,0.000024,0.000099,0.000189,0.000009,0.000023,0.000159,0.000091,0.000084,0.000018,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
13,0.000024,1.000000,0.000020,0.000022,0.000014,0.000089,0.000021,0.000019,0.000019,0.000058,...,0.000012,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011
19,0.000099,0.000020,1.000000,0.000184,0.000008,0.000019,0.000116,0.000475,0.000410,0.000015,...,0.000010,0.000010,0.000010,0.000010,0.000009,0.000010,0.000010,0.000010,0.000009,0.000009
26,0.000189,0.000022,0.000184,1.000000,0.000009,0.000021,0.000138,0.000145,0.000131,0.000016,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
29,0.000009,0.000014,0.000008,0.000009,1.000000,0.000014,0.000008,0.000008,0.000008,0.000017,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009


In [125]:
tom = pd.Series({'LOAN': 50000,
 'MORTDUE': 47000,
 'VALUE': 200000,
 'YOJ': 5,
 'DEROG': 0,
 'DELINQ': 0,
 'CLAGE': 100,
 'CLNO': 18,
 'DEBTINC': 0,
 'NINQ': 0,
 'REASON_DebtCon':1,
 'REASON_HomeImp':0,
 'JOB_Mgr':0,
 'JOB_Office':0,
 'JOB_Other':0,
 'JOB_ProfExe':0,
 'JOB_Sales':0,
 'JOB_Self':1})

data_pos['TOM'] = tom

In [127]:
model_rf.predict(pd.DataFrame(tom).T)[0]

1

In [48]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pos.T, 'euclidean'))), 
                         index=data_pos.columns, columns=data_pos.columns)


similarities = distances['TOM'].sort_values(ascending=False)[1:]

## Este es el vector de similiaridad de TOM con el resto de usuarios respecto a sus
## visualizaciones comunes

similars = pd.DataFrame(similarities)

In [110]:
cols = ['JOB_Self','JOB_Mgr','JOB_Other','JOB_Office','JOB_ProfExe',
        'REASON_DebtCon','REASON_HomeImp','DEROG','DELINQ']
bigger = ['MORTDUE','CLAGE','CLNO','DEBTINC','NINQ']
smaller = ['LOAN',]

df_empty_2 = pd.DataFrame({title:[] for title in data_pos.index})
df_empty_3 = pd.DataFrame({title:[] for title in data_pos.index})
df_empty_4 = pd.DataFrame({title:[] for title in data_pos.index})
for i in similars.index:
    if [data_pos[i][col] == tom[col] for col in cols] == [True]*len(cols):
        if [data_pos[i][col] > tom[col] for col in bigger] == [True]*len(bigger):
            if data_pos[i]['YOJ'] < tom['YOJ']:
                if data_pos[i]['LOAN'] < tom['LOAN']:
                    print('Ask for a smaller loan')
                    df_empty_3 = pd.concat([df_empty_3,pd.DataFrame(data_pos[i]).T])
                else:
                    print('You can get your loan')
                    df_empty_4 = pd.concat([df_empty_4,pd.DataFrame(data_pos[i]).T])
            else:
                print('Work longer')
                df_empty_2 = pd.concat([df_empty_2,pd.DataFrame(data_pos[i]).T])

display(df_empty_2.sort_values(by='LOAN',ascending=False))
display(df_empty_3.sort_values(by='LOAN',ascending=False))
display(df_empty_4.sort_values(by='LOAN',ascending=False))

Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Ask for a smaller loan
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer
Work longer


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self
5929,71200.0,196601.0,289430.0,12.0,0.0,0.0,226.914307,2.0,25.0,29.220351,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5928,70800.0,192044.0,292380.0,10.0,0.0,0.0,221.608486,2.0,25.0,30.378548,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5927,70500.0,196690.0,288512.0,9.0,0.0,0.0,227.027136,2.0,24.0,29.582301,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5926,70300.0,197761.0,294169.0,9.0,0.0,0.0,212.239725,2.0,24.0,28.907073,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5924,69700.0,192163.0,293901.0,10.0,0.0,0.0,222.147526,2.0,24.0,29.038236,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5925,69700.0,191972.0,294367.0,10.0,0.0,0.0,230.705613,2.0,25.0,28.201871,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5923,69600.0,198702.0,293252.0,10.0,0.0,0.0,208.299007,2.0,24.0,27.761644,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5922,69000.0,197587.0,295551.0,10.0,0.0,0.0,211.776391,3.0,24.0,30.356857,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5921,68700.0,197356.0,291242.0,10.0,0.0,0.0,217.801647,2.0,24.0,29.423058,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5920,68100.0,193298.0,288525.0,10.0,0.0,0.0,228.287295,2.0,25.0,30.395869,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self
4698,24800.0,98139.0,121696.0,3.0,0.0,0.0,208.523267,1.0,27.0,32.644265,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4658,24400.0,95843.0,123377.0,2.0,0.0,0.0,201.976756,1.0,26.0,32.903468,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4623,24200.0,90626.0,116601.0,2.0,0.0,0.0,226.116387,1.0,27.0,34.499938,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4576,23900.0,90244.0,120751.0,3.0,0.0,0.0,194.987252,1.0,26.0,35.406060,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4486,23400.0,89226.0,119604.0,3.0,0.0,0.0,221.681383,1.0,26.0,34.556201,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4484,23400.0,97865.0,120172.0,2.0,0.0,0.0,200.575752,1.0,26.0,36.398729,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4399,22900.0,95321.0,123082.0,3.0,0.0,0.0,205.441519,2.0,26.0,34.053491,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4261,22200.0,93947.0,116176.0,2.0,0.0,0.0,205.029409,2.0,26.0,34.939403,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4179,21800.0,90065.0,122335.0,3.0,0.0,0.0,212.695468,1.0,26.0,34.846919,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4188,21800.0,96709.0,123613.0,3.0,0.0,0.0,218.730333,2.0,27.0,35.874495,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,REASON_DebtCon,REASON_HomeImp,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self
